12. Load SGJP dictionary (Słownik SGJP dane tekstowe) to ElasticSearch (one document for each form) and use fuzzy matching to obtain the possible corrections of the 30 words with 5 occurrences that do not belong to the dictionary.

In [1]:
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [3]:
sqjp = '/Users/kuba/PycharmProjects/pythonProject/pjn/sgjp-20221030.text'
lines = open(sqjp, 'r', encoding='utf-8').read().split("\n")
sgjp_dict = [line.split()[0].lower() for line in lines if len(line.split())>0]

In [14]:
es = Elasticsearch(['http://localhost:9200'], http_auth=('elastic', 'Yel1*I2azvKlPYFHPj0n	'))

/var/folders/7q/26ns_sk13xs_c_vf1mhp863h0000gp/T/ipykernel_9745/223447729.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(['http://localhost:9200'], http_auth=('elastic', 'Yel1*I2azvKlPYFHPj0n	'))


In [20]:
res = es.indices.create(
    index="lab3_ind",
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "analyzer_lab3": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "morfologik_stem"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "token": {
                    "type": "text",
                    "analyzer": "analyzer_lab3"
                }
            }
        }
    })

print(res)

/var/folders/7q/26ns_sk13xs_c_vf1mhp863h0000gp/T/ipykernel_9745/413185431.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.indices.create(


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'lab3_ind'}


Ładowanie do indexu przez bulk

In [ ]:
body = []
temp = 0
with tqdm(total=len(sgjp_dict)) as pbar:
    for word in sgjp_dict:
        body.append(({'index': {}}))
        body.append({"token": word})
        #temp+=1
        pbar.update(1)
        if pbar.n % 10000 == 0:
        #if temp % 5000 == 0:
            res= es.bulk(index="lab3_ind", body=body, request_timeout=30)
            body = []

30 tych samych losowych slow

In [80]:

words_to_correct = [('wynikającez', 'wynikające'),
 ('złożeniefałszywego', 'fałszywego'),
 ('PrzewodniczącyKomitetu', 'Przewodniczącymi'),
 ('właściwemunaczelnikowi', 'właścicielowi'),
 ('przewidzianychw', 'przewidzianych'),
 ('uzyskaniuzezwolenia', 'zezwolenia'),
 ('latachsłużby', 'służby'),
 ('przepisyprawa', 'przepisywania'),
 ('iinformacji', 'informacji'),
 ('izasady', 'zasady'),
 ('zpłatności', 'płatności'),
 ('wyrobówtytoniowych', 'tytoniowych'),
 ('wyrobytytoniowe', 'tytoniowe'),
 ('organwłaściwy', 'właściwy'),
 ('odokonanie', 'dokonanie'),
 ('ustneżądanie', 'żądanie'),
 ('porozdziale', 'rozdziale'),
 ('sięorzeczenia', 'orzeczenia'),
 ('wzamian', 'zamian'),
 ('daneidentyfikujące', 'identyfikujące'),
 ('centralnejewidencji', 'centralnej'),
 ('lubd', 'lub'),
 ('zdaniewstępne', 'niedostępne'),
 ('iwpłacać', 'wpłacać'),
 ('składekjest', 'składek'),
 ('środowiskaokreśli', 'środowiskowymi'),
 ('dokońca', 'końca'),
 ('zostałyuzyskane', 'uzyskane'),
 ('iżywienia', 'żywienia'),
 ('nawykonywanie', 'wykonywanie')]

In [81]:
words_to_correct = [[el for el in i] for i in words_to_correct]
words_to_correct = sum(words_to_correct, [])
Levenshtein_distance_corrections = words_to_correct[1::2]
del words_to_correct[1::2]

In [82]:
words_to_correct

['wynikającez',
 'złożeniefałszywego',
 'PrzewodniczącyKomitetu',
 'właściwemunaczelnikowi',
 'przewidzianychw',
 'uzyskaniuzezwolenia',
 'latachsłużby',
 'przepisyprawa',
 'iinformacji',
 'izasady',
 'zpłatności',
 'wyrobówtytoniowych',
 'wyrobytytoniowe',
 'organwłaściwy',
 'odokonanie',
 'ustneżądanie',
 'porozdziale',
 'sięorzeczenia',
 'wzamian',
 'daneidentyfikujące',
 'centralnejewidencji',
 'lubd',
 'zdaniewstępne',
 'iwpłacać',
 'składekjest',
 'środowiskaokreśli',
 'dokońca',
 'zostałyuzyskane',
 'iżywienia',
 'nawykonywanie']

szukanie ich poprawnych wersji

In [77]:
corrections = []
for word in words_to_correct:
    query = {
        "query": {
            "fuzzy": {
                "token": {
                    "value": word,
                    "fuzziness": "AUTO",
                    "max_expansions": 10000,
                }
            }
        }
    }
    res = es.search(index='lab3_ind',body=query)["hits"]
    try:
        corrections.append(res['hits'][0]['_source']['token'])
    except:
        corrections.append("no correction")

/var/folders/7q/26ns_sk13xs_c_vf1mhp863h0000gp/T/ipykernel_9745/4285723088.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='lab3_ind',body=query)["hits"]


In [78]:
dictionary = dict(zip(words_to_correct, corrections))

In [79]:
dictionary

{'wynikającez': 'wynikająco',
 'złożeniefałszywego': 'no correction',
 'PrzewodniczącyKomitetu': 'no correction',
 'właściwemunaczelnikowi': 'no correction',
 'przewidzianychw': 'no correction',
 'uzyskaniuzezwolenia': 'no correction',
 'latachsłużby': 'no correction',
 'przepisyprawa': 'no correction',
 'iinformacji': 'informacja',
 'izasady': 'zasadna',
 'zpłatności': 'no correction',
 'wyrobówtytoniowych': 'no correction',
 'wyrobytytoniowe': 'no correction',
 'organwłaściwy': 'no correction',
 'odokonanie': 'dokonania',
 'ustneżądanie': 'no correction',
 'porozdziale': 'no correction',
 'sięorzeczenia': 'no correction',
 'wzamian': 'zamian',
 'daneidentyfikujące': 'no correction',
 'centralnejewidencji': 'no correction',
 'lubd': 'luba',
 'zdaniewstępne': 'no correction',
 'iwpłacać': 'niewpłacająca',
 'składekjest': 'no correction',
 'środowiskaokreśli': 'no correction',
 'dokońca': 'bokińca',
 'zostałyuzyskane': 'no correction',
 'iżywienia': 'ożywienia',
 'nawykonywanie': 'no co

13. Compare the results of your algorithm and output of ES.

In [84]:
import pandas as pd
dt = pd.DataFrame(list(zip(words_to_correct, Levenshtein_distance_corrections, corrections)), columns=['words to correct', 'LD', 'ES'])

In [85]:
dt

,words to correct,LD,ES
0,wynikającez,wynikające,wynikająco
1,złożeniefałszywego,fałszywego,no correction
2,PrzewodniczącyKomitetu,Przewodniczącymi,no correction
3,właściwemunaczelnikowi,właścicielowi,no correction
4,przewidzianychw,przewidzianych,no correction
5,uzyskaniuzezwolenia,zezwolenia,no correction
6,latachsłużby,służby,no correction
7,przepisyprawa,przepisywania,no correction
8,iinformacji,informacji,informacja
9,izasady,zasady,zasadna


14 .Draw conclusions regarding:

the number of true misspellings vs. the number of unknown words

Przy uzyciu metody z Levenshtein distance znajdywało polaczenie z najbardziej prawdopodobnym slowem dlatego nie ma zadnych slow nieznanych. W ES jest to 21 slow.

the performance of your method compared to ElasticSearch

ElasticSearch wyszukuje poprawień dużo szybciej

the results provided by your method compared to ElasticSearch

W metodzie z uzyciem Levenshtein distance wszystkie slowa znalazly jakies poprawne wersje w ES wiekszosc nie zostala poprawiona
